## Travel Planning Agentic System using LLMchain

### This project is an enhanced Travel Planning Agentic System using CrewAI by adding "real_time flight search" and "hotel search" agents.

1.Install dependencies

In [ ]:
!pip install crewai langchain langchain-openai langchain-community python-dotenv serpapi

2. Imports & Environment Setup

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from serpapi import GoogleSearch

In [ ]:
# Load API Keys
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
serpapi_api_key = os.getenv("SERPAPI_API_KEY")

if not (openain_api_key and tavily_api_key and serpapi_api_key):
    raise ValueError("Please set OPENAI_API_KEY, TAVILY_API_KEY, and SERPAPI_API_KEY in .env")

3. Initialize LLM

In [ ]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.7, openai_api_key=openai_api_key)

4. Define Web Search Tool (Tavily)

In [ ]:
# Define Web Search Tool (Tavily)
search = TavilySearchResults(api_key=tavily_api_key, max_results=5)

@tool("Web Search")
def search_web(query: str) -> str:
    results = search.invoke(query)
    return "\n\n".join(
        f"Source: {r.get('url')}\nTitle: {r.get('title')}\nContent: {r.get('content')}"
        for r in results
    )

5. Define Flight Search Tool (SerpAPI)

In [ ]:
@tool("Flight Search")
def search_flights(query: str) -> str:
    """Search real-time flights using SerpAPI."""
    params = {
        "engine": "google_flights",
        "q": query,
        "api_key": serpapi_api_key,
        "hl": "en"
    }
    search = GoogleSearch(params)
    results = search.get_dict()

    flights = results.get("flight_results", [])
    if not flights:
        return "No flights found."

    # Summarize top 3 flights
    summaries = []
    for flight in flights[:3]:
        summary = (
            f"Airline: {flight.get('airline', 'N/A')}, "
            f"From": {flight.get('origin', 'N/A')} To: {flight.get('destination', 'N/A')},
            f"Departure: {flight.get('departure_time', 'N/A)}, "
            f"Arrival: {flight.get('arrival_time', 'N/A')}, "
            f"Price: {flight.get('price', 'N/A')}"
        )
        summaries.append(summary)
    return "\n\n".join(summaries)

6. Define Hotel Search Tools (SerpAPI)

In [ ]:
@tool("Hotel Search")
def search_hotels(query: str) -> str:
    """Search real-time hotels using SerpAPI."""
    params = {
        "engine": "google_hotels",
        "q": query,
        "api_key": serpapi_api_key,
        "hl": "en"
    }
    search = GoogleSearch(params)
    results = search.get_dict()

    hotels = results.get("hotel_results", [])
    if not hotels:
        return "No hotels found."

    # Summarize top 3 hotels
    summaries = []
    for hotel in hotels[:3]:
        summary = (
            f"Hotel: {hotel.get('title', 'N/A')}, "
            f"Rating: {hotel.get('rating', 'N/A')}, "
            f"Price: {hotel.get('price', 'N/A')}, "
            f"Address: {hotel.get('address', 'N/A')}, "
        )
        summaries.append(summary)
    return "\n\n".join(summaries)

7. Define Agents

In [ ]:
# Destination Researcher
destination_researcher = Agent(
    role="Destination Researcher",
    goal="Gather essential info about the destination including safety, seasons, and culture.",
    backstory="Expert travel researcher with knowledge of global destinations.",
    tools=[search_web],
    llm=llm,
    memory=True,
    verbose=True
)

# Attractions Specialist
attractions_specialist = Agent(
    role="Attractions Specialist",
    goal="Identify best attractions and activities based on user preferences.",
    backstory="Local expert finding the best unique travel experiences.",
    tools=[search_web],
    llm=llm,
    memory=True,
    verbose=True
)

# Flight Search Agent
flight_search_agent = Agent(
    role="Flight Search Agent",
    goal="Find best real-time flights for the trip.",
    backstory="Expert in flight deals and real-time flight data.",
    tools=[search_flights],
    llm=llm,
    memory=True,
    verbose=True
)

# Hotel Search Agent
hotel_search_agent = Agent(
    role="Hotel Search Agent",
    goal="Find best hotels and accommodation options matching budget and preferences.",
    backstory="Hotel expert familiar with booking trends and deals.",
    tools=[search_hotels],
    llm=llm,
    memory=True,
    verbose=True
)

# Itinerary Planner
itinerary_planner = Agent(
    role="Itinerary Planner",
    goal="Create detailed day-by-day itinerary based on all research.",
    backstory="Experienced planner balancing activities, logistics, and rest.",
    tools=[],
    llm=llm,
    memory=True,
    verbose=True
)

# Local Guide
local_guide = Agent(
    role="Local Guide",
    goal="Provide practical local tips, culture, and safety advice.",
    backstory="Street-smart local with cultural wisdom.",
    tools=[search_web],
    llm=llm,
    memory=True,
    verbose=True
)

# Manager
manager = Agent(
    role="Travel Project Manager",
    goal="Coordinate all agents to deliver an optimal travel plan.",
    backstory="Experienced coordinator managing task flow.",
    llm=llm,
    memory=True,
    verbose=True
)

8. Define Tasks to Assign

In [ ]:
def create_tasks(destination, travel_dates, duration_days, preferences, budget):
    return [
        Task(
            description=f"Research {destination} for traveler going on {travel_dates}. Include climate, safety, transport, and culture.",
            agent=destination_researcher,
            expected_output="Destination research report."
        ),
        Task(
            description=f"Identify top attractions in {destination} based on preferences: {preferences}.",
            agent=attractions_specialist,
            expected_output="List of attractions with details."
        ),
        Task(
            description=f"Search for best flights to {destination} for travele dates {travel_dates}.",
            agent=flight_search_agent,
            expected_output="Flight options summary."
        ),
        Task(
            description=f"Search for best hotels in {destination} matching budget {budget}.",
            agent=hotel_search_agent,
            expected_output="Hotel options summary."
        ),
        Task(
            description=f"Create a day-by-day itinerrary for a {duration_days}-day trip to {destination} based on all previous outputs.",
            agent=itinerary_planner,
            expected_output="Daily itinerary plan."
        ),
        Task(
            description=f"Provide local tips and cultural advice for {destination}.",
            agent=local_guide,
            expected_output="Local guide report."
        )
    ]

9. User Input & Kickoff

In [ ]:
destination = input("Destination: ")
travel_dates = input("Travel dates (default June 1, 2023): ") or "June 1, 2023"
duration_days = int(input("Trip duration in days (default 3): ") or "3")
preferences = input("Preferences (e.g., Food & Culture): ") or "Cultural & Museums, Food & Culinary): ") or "Cultural & Museums, Food & Culinary"
budget = input("Budget level (budget/moderate/luxury): ") or "moderate"

tasks = create_tasks(destination, travel_dates, duration_days, preferences, budget)

travel_crew = Crew{
    agents=[
        destination_researcher,
        attractions_specialist,
        flight_search_agent,
        hotel_search_agent,
        itinerary_planner,
        local_guide
    ],
    task=tasks,
    verbose=True,
    manager_agent=manager,
    process=Process.hierarchical # or Process.autonomous for full autonomy 
)

results = travel_crew.kickoff()

10. Review results

In [ ]:
for task_result in results:
    print(f"\nTask: {task_result.task.description}")
    print(f"Output:\n{task_result.output}")

#### Added Flight Search Agent and Hotel Search Agent powered by SerpAPI
#### Coordinated using CrewAI